## Script para la construcción de las tipologías de continuos poblados
La tipología de continuos poblados necesita los shapes de continuos poblados sin recortar; es decir, donde la áreas de los continuos se **superpongan** unos con otros.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd 
                        
import os

##### Abrimos las bases de datos, toda se encuentran en el mismo sistema de coordenadasa

In [3]:
# Definimos la ruta de los archivos
path_d=r"C:\Users\Guillermo\Desktop/Python\01. Continuo urbano-rural"
## BD de continuos poblados por densidad
np_1=gpd.read_file(o+'\\02. Continuos poblados\\01_Conglomerado_alta_densidad.shp')
np_2=gpd.read_file(o+'\\02. Continuos poblados\\02_Conglomerado_media_densidad_sin_recortar.shp')
np_3=gpd.read_file(o+'\\02. Continuos poblados\\03_Conglomerado_baja_densidad_sin_recortar.shp')
## Verificamos el sistema de coordenadas
print(np_1.crs)
print(np_2.crs)
print(np_3.crs)

In [5]:
## Seleccionamos solo las variables que nos interesan
np_1r=np_1[np_1.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1, inplace=True)
np_2r=np_2[np_2.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1, inplace=True)
np_3r=np_3[np_3.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1, inplace=True)

# ##Renombramos la variable de identificación del continuo poblado
# np_1r.rename({'id_n':'idnp'}, axis=1, inplace=True)
# np_2r.rename({'id_n':'idnp'}, axis=1, inplace=True)
# np_3r.rename({'id_n':'idnp'}, axis=1, inplace=True)

## creamos una BD que solo contenga la variable de idnp_zona
# np_f_r=np_f[['id_contpb']].copy()
# np_f_r['zona']=np_f_r.id_contpb.str[:1]

In [6]:
np_3r.head()

,idnp,geometry
0,21.0,"POLYGON ((1000627.900 7992627.000, 1000602.900..."
1,54.0,"POLYGON ((889027.900 8043377.000, 889002.900 8..."
2,103.0,"POLYGON ((931427.900 8091977.000, 931402.900 8..."
3,246.0,"POLYGON ((871427.900 8174977.000, 871402.900 8..."
4,435.0,"POLYGON ((1035277.900 8236877.000, 1035252.900..."


##### Creamos la lista de continuos poblados que desaparecen al no tener población
Este paso solo aplica si la tipología se realiza para los continuo poblados con población. Este paso se debe omitir si se desea tener una tipología solo para los polígonos de las tres densidades independientemente de que dichos polígonos cuenten con población según el CPV 2017

In [7]:
### Cruzamos la BD originales con la base limpia para obtener los conti
np_3r1=np_3r.merge(np_f_r[np_f_r.zona=='A'], left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
np_3c=np_3r1[np_3r1._merge=='both'].copy()

np_1r1=np_1r.merge(np_f_r, left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
np_1c=np_1r1[np_1r1._merge=='both'].copy()

np_15r1=np_15r.merge(np_f_r, left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
np_15c=np_15r1[np_15r1._merge=='both'].copy()

NameError: name 'np_f_r' is not defined

In [8]:
## Nos quedamos únicamente con las dos variables que nos interesan: el código del continuo poblado y su geometría para
## los geoprocesamientos
np_1r=np_1c[['idnp','geometry']].copy()
np_2r=np_2c[['idnp','geometry']].copy()
np_3r=np_3c[['idnp','geometry']].copy()

NameError: name 'np_3c' is not defined

## Geo procesamiento
- p1: continuos poblados de 0.8 viv/ha que contienen continuos poblados de 3.9 viv ha.
- p2: continuos poblados de 0.13 viv/ha que contienen continuos poblados de 3.9 viv ha.
- p3: continuos poblados de 0.13 viv/ha que contienen continuos poblados de 0.8 viv ha.

##### Mediana densidad

In [7]:
p1 = gpd.sjoin(np_2r, np_1r, how="left", op='contains')

C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
p1[p1.duplicated('idnp_left')]
p1[p1.idnp_left==8742.0]

,idnp_left,geometry,index_right,idnp_right
87,8742.0,"POLYGON ((306477.900 8626827.000, 306452.900 8...",20.0,2942.0
87,8742.0,"POLYGON ((306477.900 8626827.000, 306452.900 8...",16.0,2694.0
87,8742.0,"POLYGON ((306477.900 8626827.000, 306452.900 8...",15.0,2669.0
87,8742.0,"POLYGON ((306477.900 8626827.000, 306452.900 8...",17.0,2705.0


In [9]:
p1 = gpd.sjoin(np_1r, np_3r, how="left", op='intersects')
### Calculo de la número de continuos poblados y de las donnas
p1['n1_3']=np.where(p1.index_right.notna(),1,0) ##toma valor 1 cuando encuentra algún polígono de densidad 3 en el polígono de densidad 1
p1=p1.groupby('idnp_left').sum().reset_index().sort_values('n1_3') ## se suma el número de continuos de densidad 3 en el polígono de densidad 1
p1['dum1_3']=np.where(p1.n1_3>=1,1,0) ## Creamos una variable dummy 1 sí continene un continuo poblado de densidad 3; 0, no contiene un continuo poblado de densidad 3

## Nos quedamos con las variables que nos interesan 
p1=p1[['idnp_left','n1_3','dum1_3']].copy()

C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


##### Baja densidad

In [10]:
##Baja densidad con alta densidad
p2 = gpd.sjoin(np_15r, np_3r, how="left", op='intersects')
### Calculo de la número de continuos poblados y de las donnas
p2['n15_3']=np.where(p2.index_right.notna(),1,0)
p2=p2.groupby('idnp_left').sum().reset_index().sort_values('n15_3')
p2['dum15_3']=np.where(p2.n15_3>=1,1,0)
p2=p2[['idnp_left','n15_3','dum15_3']].copy()

C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [11]:
##Baja densidad con media densidad
# p3 = gpd.sjoin(np_15r, np_3r, how="left", op='intersects')
# p3[p3.idnp_left==13336.0]

In [12]:
##Baja densidad con media densidad
p3 = gpd.sjoin(np_15r, np_1r, how="left", op='intersects')
## Calculo de la número de continuos poblados y de las donnas
p3['n15_1']=np.where(p3.index_right.notna(),1,0)
p3=p3.groupby('idnp_left').sum().reset_index().sort_values('n15_1')
p3['dum15_1']=np.where(p3.n15_1>=1,1,0)
p3=p3[['idnp_left','n15_1','dum15_1']].copy()

C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


##### Juntamos las posibilidades de continuos poblados de baja densidad y creamos una tipología

In [13]:
p15_f=p3.merge(p2, on='idnp_left', how='outer', validate='1:1')

In [25]:
## Creamos la tipología de continuos poblados
p15_f['tipologia']='B0'
p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1==1),'B1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1>1),'B2',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_3==1)&(p15_f.n15_3==1),'C1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_3==1)&(p15_f.n15_3>1),'C2',p15_f.tipologia)

p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1==1)&(p15_f.dum15_3==1)&(p15_f.n15_3==1),'D1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1==1)&(p15_f.dum15_3==1)&(p15_f.n15_3>1),'D2',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1>1)&(p15_f.dum15_3==1)&(p15_f.n15_3==1),'D3',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum15_1==1)&(p15_f.n15_1>1)&(p15_f.dum15_3==1)&(p15_f.n15_3>1),'D4',p15_f.tipologia)

p1['tipologia']='E0'
p1['tipologia']=np.where((p1.dum1_3==1)&(p1.n1_3==1),'E1',p1.tipologia)
p1['tipologia']=np.where((p1.dum1_3==1)&(p1.n1_3>1),'E2',p1.tipologia)

In [26]:
p15_f[p15_f.idnp_left==13336.0]

,idnp_left,n15_1,dum15_1,n15_3,dum15_3,tipologia
51794,13336.0,1,1,1,1,D1


##### Todas los continuos poblados de baja densidad con la variable de tipología
Obj: Agregar los continuos poblados de Media densidad no incluidos y agregar los continuos poblados de Alta densidad no incluidos


In [27]:
## Todos los NP de baja densidad con la variable de tipología 
np_15r2=np_15r.merge(p15_f, left_on='idnp', right_on='idnp_left',
                     how='outer', validate='1:1', indicator=True)[['idnp_left','geometry','n15_1','dum15_1','n15_3','dum15_3','tipologia']] ##match perfecto
np_1r2=np_1r.merge(p1, left_on='idnp', right_on='idnp_left',
                   how='outer', validate='1:1', indicator=True)[['idnp_left','geometry','n1_3','dum1_3','tipologia']] ##match perfecto

Hacemos un geoprocesamiento para identificar a los continuos poblados de MEDIANA densidad que se encuentran incluidos en la BD de BAJA densidad,
luego recodificamos las variable "tipología"

In [33]:
np_15r3[np_15r3.index_left.notna()]

,index_left,tipologia_left,idnp_left,geometry,n1_3,dum1_3,tipologia_right
0,101,D3,48.0,"POLYGON ((1000427.900 7992427.000, 1000402.900...",1,1,E1
1,101,D3,51.0,"POLYGON ((998477.900 7995427.000, 998452.900 7...",0,0,E0
2,204,D1,97.0,"POLYGON ((893027.900 8043127.000, 893002.900 8...",1,1,E1
3,452,D1,199.0,"POLYGON ((931527.900 8091777.000, 931502.900 8...",1,1,E1
4,524,B1,205.0,"POLYGON ((841127.900 8097927.000, 841102.900 8...",0,0,E0
...,...,...,...,...,...,...,...
289,51075,B1,25052.0,"POLYGON ((-133822.100 9589577.000, -133847.100...",0,0,E0
290,51151,B1,25095.0,"POLYGON ((-124022.100 9594777.000, -124047.100...",0,0,E0
291,51158,B1,25102.0,"POLYGON ((-107422.100 9595877.000, -107447.100...",0,0,E0
292,51201,D1,25143.0,"POLYGON ((-106322.100 9600827.000, -106347.100...",1,1,E1


In [34]:
np_15r3=gpd.sjoin(np_15r2[['tipologia','geometry']], np_1r2, how="right", op='intersects')
np_15r3['tipologia_left']=np.where(np_15r3.index_left.isna(),np_15r3.tipologia_right, np_15r3.tipologia_left)
np_15r3=np_15r3.reset_index()[['idnp_left','geometry','n1_3','dum1_3','tipologia_left']].copy()
np_15r3.rename({'tipologia_left':'tipologia'},axis=1, inplace=True)

g1=gpd.sjoin(np_15r2[['tipologia','geometry']], np_3r, how='right', op='intersects').reset_index()
g1['tipologia']=np.where(g1.index_left.isna(),'A0',g1.tipologia)
g1=g1[['idnp','geometry','tipologia']].copy()
g1.rename({'idnp':'idnp_left'},axis=1, inplace=True)


C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


##### Juntamos la BD de BAJA, MEDIA y ALTA densidad 

In [35]:
np_f1=np_15r2.append(np_15r3)
#np_ft=np_f1.append(g1)

In [36]:
##ALTA Y MEDIA DENSIDAD
np_f1=g1.append(np_15r3)
np_f1=np_f1[['idnp_left','tipologia']].copy()
##grabamos
np_f1.to_excel(o+'\\a_m_tipologia.xlsx', index=False)

##BAJA DENSIDAD
np_f2=np_15r2
np_f2=np_f2[['idnp_left','tipologia']].copy()
##grabamos
np_f2.to_excel(o+'\\baja_tipologia.xlsx', index=False)

In [37]:
np_f2.groupby('tipologia').size()

tipologia
B0    51644
B1      208
B2       11
D1       27
D3       11
D4        2
dtype: int64

In [38]:
np_f1.groupby('tipologia').size()

tipologia
B1    208
B2     29
D1     54
D3     36
D4     12
dtype: int64